### Download input files and required modules

In [ ]:
# retrieve the sp code
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the spatial queue model
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/queue_model.py" -O queue_model.py

# retrieve the road network
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_edges.csv" -O traffic_inputs/berkeley_edges.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_nodes.csv" -O traffic_inputs/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_20pn.csv" -O traffic_inputs/od_20pn.csv

# create output directory
!mkdir -p traffic_outputs
!mkdir -p traffic_outputs/t_stats
!mkdir -p traffic_outputs/node_stats
!mkdir -p traffic_outputs/link_stats

### Import modules

In [ ]:
import time 
import random
import logging 
import numpy as np
import pandas as pd 
random.seed(0)
np.random.seed(0)

### spatial queue model
from queue_model import Node, Link, Agent, Simulation

### Run the simulation
Note: if you change anything below, you need to re-run all cells start from here

In [ ]:
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
links_df = pd.read_csv('traffic_inputs/berkeley_edges.csv')
od_df = pd.read_csv('traffic_inputs/od_20pn.csv')

simulation = Simulation()
simulation.create_network(nodes_df, links_df)
simulation.create_demand(od_df)

In [ ]:
### calculating shortest path
cannot_find_path = []
for agent_id, agent in simulation.all_agents.items():
  routing_status = agent.get_path( g=simulation.g )
  if routing_status == 'n_a':
    cannot_find_path.append(agent_id)

for agent_id in cannot_find_path:
  del simulation.all_agents[agent_id]
print('# trips whose paths cannot be found: {}'.format(len(cannot_find_path)))
print('# trips {}'.format(len(simulation.all_agents)))

In [ ]:
t_start, t_end = 0, 2001
reroute_frequency = 60 ### sec

with open('traffic_outputs/t_stats/t_stats.csv', 'w') as t_stats_outfile:
  t_stats_outfile.write("t,arr"+"\n")

for t in range(t_start, t_end):
  
  ### load agents
  for agent_id, agent in simulation.all_agents.items(): 
    agent.load_trips(t)
    ### reroute
    if (t>0) and (t%reroute_frequency == 0):
      routing_status = agent.get_path( g=simulation.g )

  ### run link model
  for link_id, link in simulation.all_links.items():
    link.run_link_model(t)

  ### run node model
  node_ids_to_run = set([link.end_nid for link in simulation.all_links.values() if len(link.queue_veh)>0])
  for node_id in node_ids_to_run:
    node = simulation.all_nodes[node_id] 
    node.run_node_model(t)
  
  ### output results
  if t%100 == 0:
    arrival_cnts = np.sum([1 for a in simulation.all_agents.values() if a.status=='arr'])
    print(t, arrival_cnts)
    if arrival_cnts == len(simulation.all_agents):
        print("all agents arrive at destinations")
        break
    with open('traffic_outputs/t_stats/t_stats.csv', 'a') as t_stats_outfile:
        t_stats_outfile.write("{},{}".format(t, arrival_cnts) + "\n")
  
  if t%100 == 0:
    link_output = pd.DataFrame([(link.lid, len(link.queue_veh), len(link.run_veh), round(link.travel_time, 2)) for link in simulation.all_links.values() if link.ltype[0:2]!='vl'], columns=['link_id', 'q', 'r', 't'])
    link_output[(link_output['q']>0) | (link_output['r']>0)].reset_index(drop=True).to_csv('traffic_outputs/link_stats/link_stats_t{}.csv'.format(t), index=False)
    node_predepart = pd.DataFrame([(agent.cle, 1) for agent in simulation.all_agents.values() if (agent.status in [None, 'loaded'])], columns=['node_id', 'predepart_cnt']).groupby('node_id').agg({'predepart_cnt': np.sum}).reset_index()
    node_predepart.to_csv('traffic_outputs/node_stats/node_stats_t{}.csv'.format(t), index=False)

In [ ]:
### visualization
# nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
# links_df = pd.read_csv('traffic_inputs/berkeley_edges.csv')
for t in range(100, 1100, 100):
  link_stats = pd.read_csv('traffic_outputs/link_stats/link_stats_t{}.csv'.format(t)).merge(links_df[['link_id', 'geometry']], how='left', on='link_id').to_csv('traffic_outputs/visualization/link_stats_t{}.csv'.format(t))
  node_stats = pd.read_csv('traffic_outputs/node_stats/node_stats_t{}.csv'.format(t)).merge(nodes_df[['node_id', 'lon', 'lat']], how='left', on='node_id').to_csv('traffic_outputs/visualization/node_stats_t{}.csv'.format(t))

In [ ]:
!cd traffic_outputs && tar czvf visualization.tar.gz visualization